In [ ]:
import torch
import torch.nn as nn
from torchvision.datasets import ImageFolder
from torchvision.transforms import Compose, ToTensor, Lambda, Resize, Normalize
from torch.utils.data import DataLoader, Subset
import pandas as pd
from PIL import Image

In [ ]:
DIRECTROY = 'data'
MODEL_PATH = 'models'

In [ ]:
Image.open(f'{DIRECTROY}/train/0000e88ab.jpg').convert('RGB').show()

In [ ]:
df_train = pd.read_csv(f'{DIRECTROY}/train.csv')